In [2]:
import os
from langchain_groq import ChatGroq
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

In [3]:
groq_api_key = os.getenv("GROQ-API_KEY")

In [4]:
LLM = ChatGroq(model="mixtral-8x7b-32768", api_key=groq_api_key,temperature=0)

In [13]:
graph = Neo4jGraph(url="bolt://44.200.51.232:7687",
    username="neo4j",
    password="troops-runs-measurement",
    database = "neo4j")

In [14]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    LLM,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

cypher_chain.run("In which movies  acted Tom Hanks? return the number and a list of them")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN count(m) AS number_of_movies, collect(m.title) AS movie_titles

Full Context:
[{'number_of_movies': 12, 'movie_titles': ['Apollo 13', "You've Got Mail", 'A League of Their Own', 'Joe Versus the Volcano', 'That Thing You Do', 'The Da Vinci Code', 'Cloud Atlas', 'Cast Away', 'The Green Mile', 'Sleepless in Seattle', 'The Polar Express', "Charlie Wilson's War"]}]

> Finished chain.


'Tom Hanks has acted in a total of 12 movies, and those are: [\'Apollo 13\', "You\'ve Got Mail", \'A League of Their Own\', \'Joe Versus the Volcano\', \'That Thing You Do\', \'The Da Vinci Code\', \'Cloud Atlas\', \'Cast Away\', \'The Green Mile\', \'Sleepless in Seattle\', \'The Polar Express\', "Charlie Wilson\'s War"].'

In [15]:
cypher_chain.run("What movies did Meg Ryan act in? return the count and list of names")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Meg Ryan'
RETURN count(m) AS count, collect(m.title) AS movie_names

Full Context:
[{'count': 5, 'movie_names': ['Sleepless in Seattle', "You've Got Mail", 'Top Gun', 'When Harry Met Sally', 'Joe Versus the Volcano']}]

> Finished chain.


'Meg Ryan has acted in a total of 5 movies, including "Sleepless in Seattle", "You\'ve Got Mail", "Top Gun", "When Harry Met Sally", and "Joe Versus the Volcano".'

In [16]:
cypher_chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Top Gun'
RETURN p.name

Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, and Anthony Edwards played in Top Gun.'